from https://github.com/brilee/python_uct

In [ ]:
# main function for the Monte Carlo Tree Search from https://www.geeksforgeeks.org/ml-monte-carlo-tree-search-mcts/
def monte_carlo_tree_search(root): #root is an initialized dummy node, in our case, a smiles string of a hypothetical product
	
	while resources_left(time, computational power): 
		leaf = traverse(root) #if node.visits = 0, rollout, else node.expand()
		simulation_result = rollout(leaf) 
		backpropagate(leaf, simulation_result) 
		
	return best_child(root) 

# function for node traversal 
def traverse(node): 
	while fully_expanded(node): 
		node = best_uct(node) 
		
	# in case no children are present / node is terminal 
	return pick_univisted(node.children) or node 

# function for the result of the simulation 
def rollout(node): 
	while non_terminal(node): 
		node = rollout_policy(node) 
	return result(node) 

# function for randomly selecting a child node (the heart of MCTS)
def rollout_policy(node): 
	return pick_random(node.children) 

# function for backpropagation 
def backpropagate(node, result): 
	if is_root(node) return
	node.stats = update_stats(node, result) 
	backpropagate(node.parent) 

# function for selecting the best child 
# node with highest number of visits 
def best_child(node): 
	return max_visits(child)


In [65]:
import collections
import numpy as np
import math

class UCTNode(): # node object
    def __init__(self, state, move, parent=None):
        self.state = state # in our case a structure
        self.move = move
        self.is_expanded = False # starts with no children
        self.parent = parent  # Optional[UCTNode]
        self.children = {} # Dict[move, UCTNode] #pre intia
        self.child_priors = np.zeros([362], dtype=np.float32)
        self.child_total_value = np.zeros([362], dtype=np.float32)
        self.child_number_visits = np.zeros([362], dtype=np.float32)

    @property # number_visits getter
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter # setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value

    @property # total_value getter
    def total_value(self):
        return self.parent.child_total_value[self.move]

    @total_value.setter # setter
    def total_value(self, value):
        self.parent.child_total_value[self.move] = value

    def child_Q(self):
        return self.child_total_value / (1 + self.child_number_visits) # essentially number successes/number of visits

    def child_U(self):
        return math.sqrt(self.number_visits) * (
            self.child_priors / (1 + self.child_number_visits)) # decreases as the number of visits increases? what is priors

    def best_child(self):
        return np.argmax(self.child_Q() + self.child_U()) # of the children find the child with the highest aggregate of quality and uncertainty

    def select_leaf(self): # need to immediately rollout if not is_expanded
        current = self
    while current.is_expanded:
        best_move = current.best_child()
        current = current.maybe_add_child(best_move)
        return current

    def expand(self, child_priors):
        self.is_expanded = True
        self.child_priors = child_priors

    def maybe_add_child(self, move):
        if move not in self.children:
            self.children[move] = UCTNode(
              self.state.play(move), move, parent=self)
        return self.children[move]

    def backpropagate(self, value_estimate: float): #after rollout, calculate value of node
        current = self
    while current.parent is not None:
        current.number_visits += 1
        current.total_value += (value_estimate *
            self.state.to_play)
        current = current.parent

class DummyNode(object): #first node I think (root node?) or otherwise parentless node
    def __init__(self):
        self.parent = None
        self.child_total_value = collections.defaultdict(float)
        self.child_number_visits = collections.defaultdict(float)


def UCT_search(state, num_reads): # refer to Segler et al. 2018 Figure 
    root = UCTNode(state, move=None, parent=DummyNode()) 
    
    
    
    for _ in range(num_reads): # iterator isn't neccessary in this iteration, just need to iterate the right number of times
        leaf = root.select_leaf() # ***STEP 1***

        if leaf == root: #if visiting for the first time skip step 2
            #count +=1
            value_estimate = RolloutPhase.evaluate(leaf)
            leaf.backpropagate(value_estimate)
            return np.argmax(root.child_number_visits)

        child_priors = ExpansionPhase.evaluate(leaf.state) # NEED to figure out what child_priors
        leaf.expand(child_priors) # expand by the predicted child_priors
        value_estimate = RolloutPhase.evaluate(leaf.child_priors) # ***STEP 3*** estimate the value of the best of child_priors ?
        leaf.backpropagate(value_estimate) # ***STEP 4***
    return np.argmax(root.child_number_visits)


class ExpansionPhase():
    @classmethod
    def evaluate(self, state): # state of leaf (child) node
        return np.random.random([362])#, np.random.random()

class RolloutPhase():
    @classmethod
    def evaluate(self, child_priors):
        return np.random.random()

class State():
    def __init__(self, to_play=1):
        self.to_play = to_play

    def play(self, move):
        return State(-self.to_play)

In [66]:
num_reads = 10000
import time
tick = time.time()
UCT_search(State(), num_reads)
tock = time.time()
print("Took %s sec to run %s times" % (tock - tick, num_reads))
#import resource
#print("Consumed %sB memory" % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

Took 0.0019655227661132812 sec to run 10000 times
